In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

/tmp/ipykernel_94030/1793723643.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import shap
from lime.lime_tabular import LimeTabularExplainer
# 加载数据集
compas_df = pd.read_csv('compas-scores-two-years.csv')
iot_df = pd.read_csv('RT_IOT2022')
product_df = pd.read_csv('pricerunner_aggregate.csv')



# Download the Data

[Source](https://archive.ics.uci.edu/ml/datasets/bank+marketing)

In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import shap
from lime.lime_tabular import LimeTabularExplainer
# 加载数据集
compas_df = pd.read_csv('compas-scores-two-years.csv')
iot_df = pd.read_csv('RT_IOT2022')
product_df = pd.read_csv('pricerunner_aggregate.csv')



,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [4]:
## original data
# cat_cols = ['job', 'marital', 'education', 'default', 'housing',
#        'loan', 'contact', 'day', 'month', 'campaign',
#        'previous', 'poutcome']

# num_cols = ['age', 'balance', 'duration', 'pdays']
# target=["y"]

## compas data
cat_cols = ['sex', 'race',  'age_cat']
num_cols = ['age',  'priors_count',  'decile_score']
data['two_year_recid'] = data['two_year_recid'].map({0: 'no', 1: 'yes'})
target = ['two_year_recid']

## iot data
# cat_cols = ['proto', 'service']
# num_cols = [
#     'flow_duration', 
#     'fwd_pkts_tot', 
#     'bwd_pkts_tot', 
#     'fwd_data_pkts_tot', 
#     'bwd_data_pkts_tot', 
#     'flow_pkts_per_sec', 
#     'down_up_ratio', 
#     'fwd_header_size_tot'
# ]
# target = ['Attack_type']


## product data 
# cat_cols = ['Product Title', 'Cluster ID', 'Cluster Label', 'Category ID']
# num_cols = ['Merchant ID']
# target = ['Label']


In [5]:
train, test = train_test_split(data, stratify=data[target], test_size=0.2, random_state=42)

In [6]:


# 如果使用整个测试集来计算性能指标，同样地，去除最后一列作为特征，最后一列作为标签
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
print(y_test)

4688    yes
3382     no
2460    yes
6974    yes
6187    yes
       ... 
1501    yes
4747    yes
925      no
2327    yes
4471     no
Name: two_year_recid, Length: 1443, dtype: object


# LightGBM

In [7]:
# from lightgbm import LGBMClassifier
# from sklearn.preprocessing import OrdinalEncoder

# # LightGBM needs categorical columns encoded as integers
# train_enc = train.copy()
# test_enc = test.copy()
# for col in cat_cols:
#     enc = OrdinalEncoder(handle_unknown="use_encoded_value", encoded_missing_value=np.nan, unknown_value=np.nan)
#     train_enc[col] = enc.fit_transform(train_enc[col].values.reshape(-1,1))
#     test_enc[col] = enc.transform(test_enc[col].values.reshape(-1,1))

In [8]:
# clf = LGBMClassifier(random_state=42)
# clf.fit(train_enc.drop(columns=target[0]), train_enc[target], categorical_feature=cat_cols)
# test_pred = clf.predict(test_enc.drop(columns=target[0]))
# test_pred_proba = clf.predict_proba(test_enc.drop(columns=target[0]))

# acc = accuracy_score(test[target[0]].values, test_pred)
# loss = log_loss(test[target[0]].values, test_pred_proba)
# print(f"Acc: {acc} | LogLoss: {loss}")

# PyTorch Tabular

In [9]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
    CategoryEmbeddingModelConfig, 
    FTTransformerConfig, 
    TabNetModelConfig, 
    GatedAdditiveTreeEnsembleConfig, 
    TabTransformerConfig, 
    AutoIntConfig
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

## Common Configs    

These are common configs which can be reused. Since the datamodule is very quick, we can just stick with the high-level API

In [10]:
data_config = DataConfig(
    target=target, #target should always be a list.
    continuous_cols=num_cols,
    categorical_cols=cat_cols,
)

trainer_config = TrainerConfig(
#     auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=256,
    max_epochs=500,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    load_best=True, # After training, load the best checkpoint
)

optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

## CategoryEmbedding

In [11]:
# model_config = CategoryEmbeddingModelConfig(
#     task="classification",
#     layers="64-32",  # Number of nodes in each layer
#     activation="ReLU", # Activation between each layers
#     learning_rate = 1e-3,
#     head = "LinearHead", #Linear Head
#     head_config = head_config, # Linear Head Config
# )

# tabular_model = TabularModel(
#     data_config=data_config,
#     model_config=model_config,
#     optimizer_config=optimizer_config,
#     trainer_config=trainer_config,
# )
# tabular_model.fit(train=train)
# tabular_model.evaluate(test)


2024-03-08 18:00:26,313 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-03-08 18:00:26,330 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-03-08 18:00:26,338 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

...

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6888427138328552     │
│         test_loss         │    0.5958092212677002     │
└───────────────────────────┴───────────────────────────┘

/home/z/miniconda3/envs/xai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


[{'test_loss': 0.5958092212677002, 'test_accuracy': 0.6888427138328552}]

## GATE (Full)    

[GATE](https://arxiv.org/pdf/2207.08548.pdf) proposes two configuration, a Full (larger) model and a lite (smaller) model.

In [12]:
# model_config = GatedAdditiveTreeEnsembleConfig(
#     task="classification",
#     learning_rate = 1e-3,
#     head = "LinearHead", #Linear Head
#     head_config = head_config, # Linear Head Config
# )

# tabular_model = TabularModel(
#     data_config=data_config,
#     model_config=model_config,
#     optimizer_config=optimizer_config,
#     trainer_config=trainer_config,
# )
# tabular_model.fit(train=train)
# tabular_model.evaluate(test)

In [13]:
# # 选择测试集中的第一条记录并去除最后一列（假设最后一列是目标变量）

# test_sample = test.iloc[0, :-1].to_frame().transpose()
# test_sample


# sample_index = 97
# test_sample_features = test.iloc[sample_index, :-1].to_frame().transpose()
# test_sample_label = test.iloc[sample_index, -1]  
# prediction = tabular_model.predict(test_sample)
# # 输出预测结果
# print("Prediction:", prediction)


In [14]:
# def predict_single_instance(model, data, index):
#     # 选择一条记录并去除最后一列
#     test_sample = data.iloc[index, :-1].to_frame().transpose()
#     # 进行推理
#     prediction = model.predict(test_sample)
#     return prediction

# # 例子：预测测试集中第一条数据
# index = 0 # 可以更改为其他索引
# prediction = predict_single_instance(tabular_model, test, index)
# print(f"Prediction for instance {index}: {prediction}")


In [15]:
# predictions = tabular_model.predict(X_test)
# print(f"Predicted label: {prediction}, Actual label: {test_sample_label}")

# # predictions 包含了上面显示的预测结果，我们需要提取 'prediction' 列作为预测类别
# predicted_labels = predictions['prediction'].values

# # 计算性能指标，使用predicted_labels代替之前的predictions
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# accuracy = accuracy_score(y_test, predicted_labels)
# precision = precision_score(y_test, predicted_labels, average='macro', zero_division=0)
# recall = recall_score(y_test, predicted_labels, average='macro', zero_division=0)
# f1 = f1_score(y_test, predicted_labels, average='macro', zero_division=0)

# print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-Score: {f1}")




In [16]:
# import shap
# import pandas as pd

# def model_prediction_wrapper(x):
#     # Assuming `x` is a NumPy array, we convert it back to DataFrame as expected by your model
#     # Let's also assume `data.columns[:-1]` contains the feature names without the target label
#     features = test.columns[:-1]
#     data_df = pd.DataFrame(x, columns=features)
    
#     # Use the model to predict
#     predictions = tabular_model.predict(data_df)
    
#     # Assuming we are interested in the probability of a specific class for feature importance
#     # Let's take the probabilities of the first class as an example
#     # Adjust this based on the class you're interested in
#     class_probabilities = predictions.iloc[:, 0].values
    
#     return class_probabilities

# # Select a subset of the test data as the background distribution
# background_data = test.iloc[:, :-1].sample(100, random_state=42).values


In [17]:
# # Initialize the KernelExplainer
# explainer = shap.KernelExplainer(model_prediction_wrapper, background_data)


In [18]:
# # Selecting a single instance for explanation
# instance_to_explain = test.iloc[0, :-1].values.reshape(1, -1)

# # Calculating SHAP values
# shap_values = explainer.shap_values(instance_to_explain, nsamples=100)  # nsamples determines the number of times to re-evaluate the model

# # 假设我们关注的是第0个样本

# feature_names = test.columns[:-1]  # 获取特征名，排除了目标变量

# # 打印出特征和对应的SHAP值
# print("Feature SHAP Values for instance", sample_index)
# for name, value in zip(feature_names, shap_values[0]):
#     print(f"{name}: {value}")



In [19]:
# # 为简化，这里我们使用一个小的子集作为示例
# # 选择X_test的一个子集或全集，根据你的计算能力
# subset_X_test = X_test.sample(100, random_state=42)  # 或使用X_test直接计算全集

# # 计算这个子集的SHAP值
# shap_values_subset = explainer.shap_values(subset_X_test.values, nsamples=100)

# # 计算并打印全局特征重要性
# global_shap_values = np.abs(shap_values_subset).mean(axis=0)
# print("Global Feature Importances:")
# for name, value in zip(feature_names, global_shap_values):
#     print(f"{name}: {value}")


In [20]:
# import numpy as np

# # 假设shap_values_subset是我们计算得到的针对一个子集的SHAP值数组
# # 且shap_values_subset.shape = (num_instances, num_features)

# def compute_stability_score(shap_values):
#     num_instances, num_features = shap_values.shape
#     stability_scores = np.zeros(num_features)
    
#     for d in range(num_features):
#         diff_sum = 0
#         count = 0
#         for i in range(num_instances):
#             for j in range(i + 1, num_instances):
#                 # 计算特征d对于实例对(i, j)的SHAP值差异
#                 diff = np.abs(shap_values[i, d] - shap_values[j, d])
#                 diff_sum += diff
#                 count += 1
#         # 计算特征d的平均稳定性分数
#         stability_scores[d] = diff_sum / count if count > 0 else 0

#     # 返回所有特征的稳定性分数的平均值作为最终的稳定性指标
#     overall_stability_score = np.mean(stability_scores)
#     return overall_stability_score

# # 计算并打印稳定性分数
# overall_stability_score = compute_stability_score(shap_values_subset)
# print(f"Overall Stability Score: {overall_stability_score}")


In [21]:
print(X_test.dtypes)
print('-----------------')
print(y_test.dtypes)



id                           int64
name                        object
first                       object
last                        object
compas_screening_date       object
sex                         object
dob                         object
age                          int64
age_cat                     object
race                        object
juv_fel_count                int64
decile_score                 int64
juv_misd_count               int64
juv_other_count              int64
priors_count                 int64
days_b_screening_arrest    float64
c_jail_in                   object
c_jail_out                  object
c_case_number               object
c_offense_date              object
c_arrest_date               object
c_days_from_compas         float64
c_charge_degree             object
c_charge_desc               object
is_recid                     int64
r_case_number               object
r_charge_degree             object
r_days_from_arrest         float64
r_offense_date      

In [22]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from lime.lime_tabular import LimeTabularExplainer

# # 假定data为Pandas DataFrame，已加载目标数据集
# # cat_cols为类别特征列名列表，num_cols为数值特征列名列表
# # target为目标变量列名列表

# # 数据预处理和模型训练的管道
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), num_cols),
#         ('cat', OneHotEncoder(), cat_cols)
#     ])

# model = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', RandomForestClassifier(random_state=42))
# ])

# model.fit(X_train, y_train)

# # 定义模型预测函数包装器，以便与LIME兼容
# def model_predict_proba_lime(data):
#     # 将数据转换为模型训练时的格式
#     if data.ndim == 1:
#         data = data.reshape(1, -1)
#     return model.predict_proba(data)

# categorical_features_indices = [X_train.columns.get_loc(col) for col in cat_cols if col in X_train.columns]

# explainer = LimeTabularExplainer(
#     X_train.values, 
#     feature_names=X_train.columns.tolist(), 
#     class_names=np.unique(y_train).tolist(),  # 将目标变量的唯一值作为类别名称
#     categorical_features=categorical_features_indices,
#     mode='classification'
# )

# # 选择一个实例来解释
# instance_index = 1  # 可以根据需要选择不同的索引
# instance = X_test.iloc[instance_index].values

# # 定义一个模型预测的包装函数，使其能够被LIME使用
# def model_predict_proba_lime(data):
#     if data.ndim == 1:
#         data = data.reshape(1, -1)
#     # 假设tabular_model是您的训练好的模型
#     # 请根据您的模型修改此行以适应模型预测方法的调用
#     predictions = tabular_model.predict_proba(data)
#     return predictions

# # 使用LIME解释器生成解释
# exp = explainer.explain_instance(
#     data_row=instance, 
#     predict_fn=model_predict_proba_lime,  # 使用包装器函数
#     num_features=X_train.shape[1]  # 指定要显示的特征数量
# )

# # 打印特征重要性
# print("Feature importances for instance", instance_index)
# for feature, importance in exp.as_list():
#     print(f"{feature}: {importance}")

## GATE (Lite)

In [23]:
# model_config = GatedAdditiveTreeEnsembleConfig(
#     task="classification",
#     learning_rate = 1e-3,
#     head = "LinearHead", #Linear Head
#     head_config = head_config, # Linear Head Config
#     gflu_stages=4,
#     num_trees=30,
#     tree_depth=5,
#     chain_trees=False
# )

# tabular_model = TabularModel(
#     data_config=data_config,
#     model_config=model_config,
#     optimizer_config=optimizer_config,
#     trainer_config=trainer_config,
# )
# tabular_model.fit(train=train)
# tabular_model.evaluate(test)

## FT Transformer

[Paper](https://arxiv.org/abs/2106.11959)

In [24]:
model_config = FTTransformerConfig(
    task="classification",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)
tabular_model.fit(train=train)
tabular_model.evaluate(test)

2024-03-08 18:00:33,747 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-03-08 18:00:33,766 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-03-08 18:00:33,775 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

...

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6930006742477417     │
│         test_loss         │    0.5965560078620911     │
└───────────────────────────┴───────────────────────────┘

/home/z/miniconda3/envs/xai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


[{'test_loss': 0.5965560078620911, 'test_accuracy': 0.6930006742477417}]

## TabTransformer    

[Paper](https://arxiv.org/abs/2012.06678)

In [25]:
model_config = TabTransformerConfig(
    task="classification",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)
tabular_model.fit(train=train)
tabular_model.evaluate(test)

2024-03-08 18:00:40,768 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-03-08 18:00:40,786 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-03-08 18:00:40,800 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

...

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6791406869888306     │
│         test_loss         │    0.6016159057617188     │
└───────────────────────────┴───────────────────────────┘

/home/z/miniconda3/envs/xai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


[{'test_loss': 0.6016159057617188, 'test_accuracy': 0.6791406869888306}]

In [30]:
# 选择测试集中的第一条记录并去除最后一列（假设最后一列是目标变量）

test_sample = test.iloc[0, :-1].to_frame().transpose()

test_sample


sample_index = 10

test_sample_features = test.iloc[sample_index, :-1].to_frame().transpose()

test_sample_label = test.iloc[sample_index, -1]  

prediction = tabular_model.predict(test_sample)
prediction


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

,no_probability,yes_probability,prediction
4688,0.644139,0.355861,no


In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from scipy.spatial.distance import euclidean
from math import tanh

# 假设已经有训练好的模型 tabular_model 和数据 X_test, y_test
def mask_feature(X, feature_name, is_categorical):
    """Mask a feature in the dataset."""
    X_masked = X.copy()
    if is_categorical:
        # Option 1: Set a common "unknown" value for categorical features
        # X_masked[feature_name] = "unknown"
        # Option 2: Replace with the mode (most common value) of the column
        mode_value = X[feature_name].mode()[0]
        X_masked[feature_name] = mode_value
    else:
        # Replace numerical features with the mean of the column
        mean_value = X[feature_name].mean()
        X_masked[feature_name] = mean_value
    return X_masked

def predict_proba_with_model(model, X):
    """
    This function is adapted for PyTorch Tabular models to predict probabilities.
    You might need to adjust it based on how your model's prediction method works.
    """
    # Ensure X is in the correct format (DataFrame for PyTorch Tabular)
    if isinstance(X, pd.DataFrame):
        predictions = model.predict(X)
    else:
        X_df = pd.DataFrame(X, columns=num_cols + cat_cols) # Adjust based on your actual feature columns
        predictions = model.predict(X_df)
    
    # Assuming the output is in a DataFrame with probability columns, adjust as necessary
    return predictions[['no_probability', 'yes_probability']].values

def compute_prediction_difference(model, X, feature_names):
    """
    Compute prediction differences when masking features.
    """
    pred_diffs = []
    base_predictions = predict_proba_with_model(model, X)
    
    for feature_name in feature_names:
        X_masked = X.copy()
        # You might want to adjust the masking strategy based on the nature of your features
        if feature_name in cat_cols:
            X_masked[feature_name] = np.random.choice(X[feature_name])
        else:
            X_masked[feature_name] = 0  # For numeric features, consider using mean or median
            
        masked_predictions = predict_proba_with_model(model, X_masked)
        
        delta = np.abs(masked_predictions - base_predictions)
        pred_diffs.append(delta)
        
    return np.array(pred_diffs)

def compute_feature_importance(pred_diffs, feature_names):
    """
    Calculate the importance for each feature based on the prediction differences.
    """
    # Calculate the mean of prediction differences for each feature
    mean_pred_diffs = np.mean(pred_diffs, axis=1)
    
    # Ranking features based on the mean prediction difference
    feature_importance = {feature: np.mean(diff) for feature, diff in zip(feature_names, mean_pred_diffs)}
    
    # Sort features by importance
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_features

# Combine categorical and numerical features for the processing
all_features = num_cols + cat_cols

# Compute prediction differences
pred_diffs = compute_prediction_difference(tabular_model, X_test, all_features)

# Compute feature importance
feature_importances = compute_feature_importance(pred_diffs, all_features)

print("Feature importances:", feature_importances)


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Feature importances: [('decile_score', 0.10836089), ('age', 0.10170803), ('age_cat', 0.08789513), ('priors_count', 0.08179231), ('race', 0.029716618), ('sex', 0.012345597)]


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

In [34]:
import numpy as np

def compute_global_feature_importance(pred_diffs, feature_names):
    """
    Compute global feature importances based on prediction differences.
    pred_diffs: A list of arrays containing prediction differences for each feature.
    feature_names: List of feature names corresponding to pred_diffs.
    """
    # Calculate mean of absolute prediction differences for each feature
    global_importances = np.mean(np.abs(pred_diffs), axis=2).mean(axis=1)
    feature_importance_dict = dict(zip(feature_names, global_importances))
    
    # Sort features by their importance
    sorted_importances = sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)
    
    return sorted_importances

def compute_stability_score(pred_diffs):
    """
    Compute stability score for the model based on prediction differences.
    pred_diffs: A list of arrays containing prediction differences for each feature.
    """
    num_samples = pred_diffs.shape[2]
    num_features = pred_diffs.shape[0]
    
    stability_scores = np.zeros(num_features)
    
    for feature_index in range(num_features):
        diffs = pred_diffs[feature_index]
        # Calculate pairwise differences for each sample
        pairwise_diffs = np.abs(diffs[:, None, :] - diffs[None, :, :])
        # Average over all pairs and then over all samples
        stability_scores[feature_index] = np.mean(pairwise_diffs)
    
    # The overall stability score is the mean of stability scores across all features
    overall_stability_score = np.mean(stability_scores)
    
    return overall_stability_score

# Assuming pred_diffs is already computed as described in the initial problem
# Example usage:
feature_names = num_cols + cat_cols  # Assuming this is your list of feature names
global_feature_importances = compute_global_feature_importance(pred_diffs, feature_names)
overall_stability_score = compute_stability_score(pred_diffs)

print("Global Feature Importances:")
for feature, importance in global_feature_importances:
    print(f"{feature}: {importance}")

print(f"Overall Stability Score: {overall_stability_score}")


Global Feature Importances:
decile_score: 0.1083609089255333
age: 0.10170802474021912
age_cat: 0.08789518475532532
priors_count: 0.08179234713315964
race: 0.02971658483147621
sex: 0.012345596216619015
Overall Stability Score: 0.05732312270750602


In [26]:
# 选择测试集中的第一条记录并去除最后一列（假设最后一列是目标变量）

test_sample = test.iloc[0, :-1].to_frame().transpose()

test_sample


sample_index = 97

test_sample_features = test.iloc[sample_index, :-1].to_frame().transpose()

test_sample_label = test.iloc[sample_index, -1]  

prediction = tabular_model.predict(test_sample)

# 输出预测结果

print("Prediction:", prediction)

def predict_single_instance(model, data, index):

    # 选择一条记录并去除最后一列

    test_sample = data.iloc[index, :-1].to_frame().transpose()

    # 进行推理

    prediction = model.predict(test_sample)

    return prediction

# 例子：预测测试集中第一条数据

index = 0 # 可以更改为其他索引

prediction = predict_single_instance(tabular_model, test, index)

print(f"Prediction for instance {index}: {prediction}")

predictions = tabular_model.predict(X_test)

print(f"Predicted label: {prediction}, Actual label: {test_sample_label}")

# predictions 包含了上面显示的预测结果，我们需要提取 'prediction' 列作为预测类别

predicted_labels = predictions['prediction'].values

# 计算性能指标，使用predicted_labels代替之前的predictions

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, predicted_labels)

precision = precision_score(y_test, predicted_labels, average='macro', zero_division=0)

recall = recall_score(y_test, predicted_labels, average='macro', zero_division=0)

f1 = f1_score(y_test, predicted_labels, average='macro', zero_division=0)

print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-Score: {f1}")

import shap

import pandas as pd

def model_prediction_wrapper(x):

    # Assuming `x` is a NumPy array, we convert it back to DataFrame as expected by your model

    # Let's also assume `data.columns[:-1]` contains the feature names without the target label

    features = test.columns[:-1]

    data_df = pd.DataFrame(x, columns=features)

    # Use the model to predict

    predictions = tabular_model.predict(data_df)

    # Assuming we are interested in the probability of a specific class for feature importance

    # Let's take the probabilities of the first class as an example

    # Adjust this based on the class you're interested in

    class_probabilities = predictions.iloc[:, 0].values

    return class_probabilities

# Select a subset of the test data as the background distribution

background_data = test.iloc[:, :-1].sample(100, random_state=42).values

# Initialize the KernelExplainer

explainer = shap.KernelExplainer(model_prediction_wrapper, background_data)

# Selecting a single instance for explanation

instance_to_explain = test.iloc[0, :-1].values.reshape(1, -1)

# Calculating SHAP values

shap_values = explainer.shap_values(instance_to_explain, nsamples=100)  # nsamples determines the number of times to re-evaluate the model

# 假设我们关注的是第0个样本

feature_names = test.columns[:-1]  # 获取特征名，排除了目标变量

# 打印出特征和对应的SHAP值

print("Feature SHAP Values for instance", sample_index)

for name, value in zip(feature_names, shap_values[0]):

    print(f"{name}: {value}")

# 为简化，这里我们使用一个小的子集作为示例

# 选择X_test的一个子集或全集，根据你的计算能力

subset_X_test = X_test.sample(100, random_state=42)  # 或使用X_test直接计算全集

# 计算这个子集的SHAP值

shap_values_subset = explainer.shap_values(subset_X_test.values, nsamples=100)

# 计算并打印全局特征重要性

global_shap_values = np.abs(shap_values_subset).mean(axis=0)

print("Global Feature Importances:")

for name, value in zip(feature_names, global_shap_values):

    print(f"{name}: {value}")

import numpy as np

# 假设shap_values_subset是我们计算得到的针对一个子集的SHAP值数组

# 且shap_values_subset.shape = (num_instances, num_features)

def compute_stability_score(shap_values):

    num_instances, num_features = shap_values.shape

    stability_scores = np.zeros(num_features)

    for d in range(num_features):

        diff_sum = 0

        count = 0

        for i in range(num_instances):

            for j in range(i + 1, num_instances):

                # 计算特征d对于实例对(i, j)的SHAP值差异

                diff = np.abs(shap_values[i, d] - shap_values[j, d])

                diff_sum += diff

                count += 1

        # 计算特征d的平均稳定性分数

        stability_scores[d] = diff_sum / count if count > 0 else 0

    # 返回所有特征的稳定性分数的平均值作为最终的稳定性指标

    overall_stability_score = np.mean(stability_scores)

    return overall_stability_score

# 计算并打印稳定性分数

overall_stability_score = compute_stability_score(shap_values_subset)

print(f"Overall Stability Score: {overall_stability_score}")

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Prediction:       no_probability  yes_probability prediction
4688        0.644139         0.355861         no
Prediction for instance 0:       no_probability  yes_probability prediction
4688        0.644139         0.355861         no
Predicted label:       no_probability  yes_probability prediction
4688        0.644139         0.355861         no, Actual label: no
Accuracy: 0.6791406791406791
Precision: 0.6758591644204852
Recall: 0.6721437578814629
F1-Score: 0.672983708141524


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

  0%|          | 0/1 [00:00<?, ?it/s]

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Feature SHAP Values for instance 97
id: 0.0
name: 0.0
first: 0.0
last: 0.0
compas_screening_date: 0.0
sex: -0.011022554281014734
dob: 0.0
age: 0.03760018861871631
age_cat: 0.0855360002348123
race: -0.009826753314111597
juv_fel_count: 0.0
decile_score: -0.007965133927387145
juv_misd_count: 0.0
juv_other_count: 0.0
priors_count: 0.04304435671797488
days_b_screening_arrest: -3.9413368682593966e-05
c_jail_in: 0.0
c_jail_out: 0.0
c_case_number: 0.0
c_offense_date: 0.0
c_arrest_date: 0.0
c_days_from_compas: 0.0
c_charge_degree: 0.0
c_charge_desc: 0.0
is_recid: -5.860230043447711e-05
r_case_number: -0.0001216668357780886
r_charge_degree: 0.0
r_days_from_arrest: 8.737550958697942e-05
r_offense_date: 0.0
r_charge_desc: 0.0
r_jail_in: 0.0
r_jail_out: 0.0
violent_recid: 0.0
is_violent_recid: 0.0
vr_case_number: 0.0
vr_charge_degree: 0.0
vr_offense_date: 0.0
vr_charge_desc: 0.0
type_of_assessment: 0.0
decile_score.1: 0.0
score_text: -0.00010115155899883312
screening_date: 0.0
v_type_of_assessment:

  0%|          | 0/100 [00:00<?, ?it/s]

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Global Feature Importances:
id: 0.00013616904845543227
name: 9.423636967038955e-05
first: 0.0001845361078616724
last: 0.0002068609241086027
compas_screening_date: 6.259082375748986e-05
sex: 0.02087371897307476
dob: 0.00015114471546222755
age: 0.028452938800777924
age_cat: 0.03907597315103123
race: 0.012227778881181482
juv_fel_count: 0.0
decile_score: 0.05711938074063934
juv_misd_count: 9.300841192510732e-05
juv_other_count: 0.00011431378369228549
priors_count: 0.10053038878716578
days_b_screening_arrest: 4.7007180098558455e-05
c_jail_in: 9.440083002984457e-05
c_jail_out: 0.0001440811895871379
c_case_number: 0.00010111820627178346
c_offense_date: 0.00018972836256324905
c_arrest_date: 9.987626619388351e-05
c_days_from_compas: 0.00011422888080360126
c_charge_degree: 0.0001129321293349604
c_charge_desc: 0.00015993095943081035
is_recid: 0.00011199878061906385
r_case_number: 6.935098073337321e-05
r_charge_degree: 0.00013384727094862528
r_days_from_arrest: 7.301369585029195e-05
r_offense_date

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Assuming `data` is your DataFrame and `target` defines the label column name
X = data.drop(columns=target)
y = data[target]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(), cat_cols)
    ])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)
# This function needs to take raw data, preprocess it, and then make a prediction
# Adjusted prediction function to handle raw input
def lime_prediction_function(raw_data):
    # Reshape raw_data if it's a Series to ensure it's in DataFrame format
    if isinstance(raw_data, pd.Series):
        raw_data = raw_data.to_frame().T
    # Apply preprocessing
    preprocessed_data = preprocessor.transform(raw_data)
    # Ensure the model's predict method is called correctly and return the prediction
    # The prediction should be returned as a 2D numpy array [[prob_class_0, prob_class_1, ..., prob_class_N]]
    predictions = tabular_model.predict_proba(preprocessed_data)
    return predictions



from lime.lime_tabular import LimeTabularExplainer

# Initialize the LIME Explainer
# Assuming cat_cols is a list of column names for categorical features in X_train
categorical_features_indices = [X_train.columns.get_loc(col) for col in cat_cols]

explainer = LimeTabularExplainer(
    training_data=X_train.to_numpy(),
    feature_names=X_train.columns.tolist(),
    class_names=[str(x) for x in np.unique(y_train)],
    categorical_features=categorical_features_indices,
    mode='classification'  # or 'regression'
)

# Ensure lime_prediction_function preprocesses data internally
def lime_prediction_function(raw_data):
    # Convert raw_data to DataFrame if it's not already
    if not isinstance(raw_data, pd.DataFrame):
        raw_data_df = pd.DataFrame(raw_data, columns=X_train.columns)
    else:
        raw_data_df = raw_data

    # Apply preprocessing here. This is just a placeholder.
    # You need to replicate the preprocessing steps applied during model training
    preprocessed_data = your_preprocessing_function(raw_data_df)

    # Obtain predictions from your model
    predictions = tabular_model.predict(preprocessed_data)

    # Return the predictions in the format expected by LIME
    return predictions

# Select a raw instance to explain
instance = X_test.iloc[0]  # This is a Series

# Generate an explanation
exp = explainer.explain_instance(
    data_row=instance.values,  # Convert Series to numpy array
    predict_fn=lime_prediction_function,
    num_features=len(X_train.columns)
)

# Visualize the explanation
exp.show_in_notebook(show_table=True)




TypeError: unsupported operand type(s) for -: 'str' and 'str'

## AutoInt    

[Paper](https://arxiv.org/abs/1810.11921)

In [ ]:
# model_config = AutoIntConfig(
#     task="classification",
#     learning_rate = 1e-3,
#     head = "LinearHead", #Linear Head
#     head_config = head_config, # Linear Head Config
# )

# tabular_model = TabularModel(
#     data_config=data_config,
#     model_config=model_config,
#     optimizer_config=optimizer_config,
#     trainer_config=trainer_config,
# )
# tabular_model.fit(train=train)
# tabular_model.evaluate(test)

## TabNet    

[Paper](https://arxiv.org/abs/1908.07442)

In [ ]:
model_config = TabNetModelConfig(
    task="classification",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)
tabular_model.fit(train=train)
tabular_model.evaluate(test)

2024-03-08 14:43:16,016 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-03-08 14:43:16,031 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-03-08 14:43:16,041 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

...

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6763686537742615     │
│         test_loss         │    0.6108263731002808     │
└───────────────────────────┴───────────────────────────┘

/home/z/miniconda3/envs/xai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


[{'test_loss': 0.6108263731002808, 'test_accuracy': 0.6763686537742615}]